In [82]:
# Set up dependencies   
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.callbacks import LambdaCallback
import tensorflow as tf
import pandas as pd 

In [83]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [84]:
# Drop the 'EIN' and 'NAME' columns.
colums_to_drop = ["EIN", "NAME"]    
application_df = application_df.drop(colums_to_drop, axis=1)
application_df.head()   

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [85]:
# Determine the number of unique values in each column.
application_df.nunique(axis=0)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [86]:
# Look at APPLICATION_TYPE value counts for binning
application_type = application_df.APPLICATION_TYPE.value_counts()
print(application_type)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


## Optimization Attempt 1
Change cut off from 250 to 800    

In [87]:
# Cut off at 800 and create a list of application types to be replaced
cutoff_value_app = 800
application_types_to_replace = application_type[application_type < cutoff_value_app].index.tolist()

# Replace with "Other" in the dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure binning was successful
print(application_df['APPLICATION_TYPE'].value_counts())

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64


In [88]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [89]:
# Cut off at 1000 and create a list of classifications to be replaced
cut_off_value_class = 1000  
classifications_to_replace = classification_counts[classification_counts < cut_off_value_class].index.tolist()  

# Replace with "Other" in the dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()  

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [90]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df)

# Display the resulting DataFrame
application_df_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [91]:
# Split the preprocessed data into target and features arrays   
y = application_df_dummies["IS_SUCCESSFUL"].values
X = application_df_dummies.drop(["IS_SUCCESSFUL"], axis=1).values   

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [92]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler with the training data
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [93]:
number_input_features = len(X_train[0])
print(number_input_features)

40


## Optimization attempt 1 
Add two hidden layers 

Increase neurons in each layer 

Change activation functions 

In [94]:
optimize_nn = tf.keras.models.Sequential()  
# First hidden layer 
optimize_nn.add(tf.keras.layers.Dense(units=128, input_dim=number_input_features, activation='relu'))   

# Second hidden layer 
optimize_nn.add(tf.keras.layers.Dense(units=128, activation='sigmoid'))

# Third hidden layer 
optimize_nn.add(tf.keras.layers.Dense(units=128, activation='tanh'))

# Fourth hidden layer 
optimize_nn.add(tf.keras.layers.Dense(units=128, activation='relu'))


# Output layer 
optimize_nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
optimize_nn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (None, 128)               5248      
                                                                 
 dense_46 (Dense)            (None, 128)               16512     
                                                                 
 dense_47 (Dense)            (None, 128)               16512     
                                                                 
 dense_48 (Dense)            (None, 128)               16512     
                                                                 
 dense_49 (Dense)            (None, 1)                 129       
                                                                 
Total params: 54913 (214.50 KB)
Trainable params: 54913 (214.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [95]:
# Compile the model
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
optimize_nn.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [96]:
# Train the model
optimize_nn.fit(X_train_scaled, y_train, epochs=100, callbacks=None)

Epoch 1/100


804/804 [==============================] - 1s 802us/step - loss: 0.5780 - accuracy: 0.7171
Epoch 2/100
804/804 [==============================] - 1s 789us/step - loss: 0.5610 - accuracy: 0.7294
Epoch 3/100
804/804 [==============================] - 1s 732us/step - loss: 0.5566 - accuracy: 0.7307
Epoch 4/100
804/804 [==============================] - 1s 757us/step - loss: 0.5559 - accuracy: 0.7324
Epoch 5/100
804/804 [==============================] - 1s 881us/step - loss: 0.5536 - accuracy: 0.7319
Epoch 6/100
804/804 [==============================] - 1s 935us/step - loss: 0.5525 - accuracy: 0.7330
Epoch 7/100
804/804 [==============================] - 1s 740us/step - loss: 0.5519 - accuracy: 0.7324
Epoch 8/100
804/804 [==============================] - 1s 830us/step - loss: 0.5515 - accuracy: 0.7333
Epoch 9/100
804/804 [==============================] - 1s 867us/step - loss: 0.5505 - accuracy: 0.7337
Epoch 10/100
804/804 [==============================] - 1s 889us/step - loss: 0.5496 

In [97]:
# Evaluate the model using the test data    
model_loss, model_accuracy = optimize_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5912 - accuracy: 0.7254 - 149ms/epoch - 557us/step
Loss: 0.5912008285522461, Accuracy: 0.7253644466400146


## Attempt 2
Increase the number of neurons in the hidden layers 

Increase the number of epochs 

Use relu for the hidden layers and sigmoid for the output layer  

In [98]:
optimize_nn2 = tf.keras.models.Sequential()  
# First hidden layer 
optimize_nn2.add(tf.keras.layers.Dense(units=200, input_dim=number_input_features, activation='relu'))   

# Second hidden layer 
optimize_nn2.add(tf.keras.layers.Dense(units=200, activation='relu'))


# Third hidden layer 
optimize_nn2.add(tf.keras.layers.Dense(units=200, activation='relu'))
optimize_nn.add(tf.keras.layers.Dropout(0.2))  # Add dropout with 30% probability

# Fourth hidden layer 
optimize_nn2.add(tf.keras.layers.Dense(units=200, activation='relu'))
optimize_nn2.add(tf.keras.layers.Dropout(0.2))  # Add dropout with 30% probability

# Output layer 
optimize_nn2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
optimize_nn2.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_50 (Dense)            (None, 200)               8200      
                                                                 
 dense_51 (Dense)            (None, 200)               40200     
                                                                 
 dense_52 (Dense)            (None, 200)               40200     
                                                                 
 dense_53 (Dense)            (None, 200)               40200     
                                                                 
 dropout_17 (Dropout)        (None, 200)               0         
                                                                 
 dense_54 (Dense)            (None, 1)                 201       
                                                                 
Total params: 129001 (503.91 KB)
Trainable params: 12

In [99]:
# Compile the model
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
optimize_nn2.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [102]:
# Train the model
optimize_nn2.fit(X_train_scaled, y_train, epochs=200, callbacks=None)

Epoch 1/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5365 - accuracy: 0.7407
Epoch 2/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5356 - accuracy: 0.7410
Epoch 3/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5351 - accuracy: 0.7405
Epoch 4/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5359 - accuracy: 0.7400
Epoch 5/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5362 - accuracy: 0.7413
Epoch 6/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5364 - accuracy: 0.7410
Epoch 7/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5381 - accuracy: 0.7384
Epoch 8/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5359 - accuracy: 0.7372
Epoch 9/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5365 - accuracy: 0.7366
Epoch 10/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5458 - accura

In [104]:
# Evaluate the model using the test data    
model_loss, model_accuracy = optimize_nn2.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 1.6771 - accuracy: 0.7219 - 127ms/epoch - 474us/step
Loss: 1.6770503520965576, Accuracy: 0.7218658924102783


## Use keras tuner to try to find the best hyperparameters for the model   

In [ ]:
# Set up keras tuner
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch


def build_model(hp):
    tuner_model = keras.Sequential()

    # Choose the number of layers
    for i in range(hp.Int('num_layers', min_value=1, max_value=5)):
        # Choose the number of neurons in each layer
        tuner_model.add(layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32), activation='relu'))
    
    tuner_model.add(layers.Dense(1, activation='sigmoid'))

    tuner_model.compile(optimizer=keras.optimizers.legacy.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return tuner_model

# Initialize the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10
)

# Search for the best hyperparameter configuration
tuner.search(X_train_scaled, y_train, epochs=100, validation_data=(X_train_scaled, y_train))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the final model with the best hyperparameters
final_model = tuner.hypermodel.build(best_hps)
